In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

np.random.seed(0)

warnings.filterwarnings('ignore')
%matplotlib inline

Реализовать класс LinearRegressionSGD c обучением и применением линейной регрессии, построенной с помощью стохастического градиентного спуска, с заданным интерфейсом

In [42]:
from sklearn.base import BaseEstimator

class LinearRegressionSGD(BaseEstimator):
    def __init__(self, epsilon=1e-4, max_steps=100, w0=None, alpha=1e-4):
        """
        epsilon: минимальная разница для нормы изменения весов
        max_steps: максимальное количество итераций
        w0: np.array (d,) - начальные веса
        alpha: шаг обучения
        """
        self.epsilon = epsilon
        self.max_steps = max_steps
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.w_history = []

    def fit(self, X, y):
        X = np.hstack([np.ones((X.shape[0], 1)), X])  # добавим bias
        l, d = X.shape
        
        # Инициализация весов
        if self.w0 is None:
            self.w = np.zeros(d)
        else:
            self.w = self.w0.copy()
        
        self.w_history.append(self.w.copy())
        
        for step in range(self.max_steps):
            # выбираем случайный индекс (stochastic gradient descent)
            i = np.random.randint(0, l)
            Xi = X[i]
            yi = y[i]
            
            # вычисляем градиент
            gradient = self.calc_gradient(Xi, yi)
            
            # обновляем веса
            new_w = self.w - self.alpha * gradient
            self.w_history.append(new_w.copy())
            
            # проверка сходимости
            if np.linalg.norm(new_w - self.w) < self.epsilon:
                self.w = new_w
                break
            self.w = new_w
        
        return self

    def calc_gradient(self, Xi, yi):
        """
        Xi: np.array (d,) — одна выборка (уже с 1 для bias)
        yi: float — истинное значение
        output: np.array (d)
        """
        y_pred = Xi @ self.w
        gradient = -2 * Xi * (yi - y_pred)
        return gradient

    def predict(self, X):
        X = np.hstack([np.ones((X.shape[0], 1)), X])  # добавим bias
        return X @ self.w

Проверять работу буду на имеющемся в sklearn наборе данных <br>Возьму стандартный [датасет](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) со стоимостью жилья в различных районах Калифорнии в 1990 году <br>Датасет содержит информацию о средних ценах на жилье в районе и какие-то параметры района: средний возраст домов, среднее число комнат, население

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing(as_frame=True)
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [44]:
X_train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
17853,5.3994,23.0,5.019157,1.022989,910.0,3.486590,37.44,-121.88
15963,3.9567,52.0,5.173664,1.127863,1848.0,3.526718,37.71,-122.44
20106,3.0500,17.0,5.383764,1.095941,753.0,2.778598,37.94,-120.29
15525,2.2500,16.0,4.331113,1.109420,2737.0,2.604186,33.14,-117.05
5234,2.0187,39.0,4.876068,1.102564,1313.0,5.611111,33.94,-118.23


Часть 1 <br>Метрикой качества в задаче будет MAPE - Mean Absolute Percentage Error <br>Реализовать её с заданным интефейсом и вычислить MAPE(y_test, y_0), где y_0 = (mean(y_test), mean(y_test), ...)

In [49]:
def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    eps = 1e-8
    return np.mean(np.abs((y_true - y_pred) / (y_true + eps))) * 100

In [50]:
y_0 = np.full_like(y_test, y_test.mean())
print("MAPE для среднего предсказания:", MAPE(y_test, y_0))

MAPE для среднего предсказания: 62.2120840858977


Часть 2 <br>Обучите LinearRegressionSGD с базовыми параметрами на тренировочном наборе данных (X_train, y_train), сделайте предсказание на тестовых данных X_test, записав результат в переменную y_pred_sgd и вычислите ошибку MAPE

In [55]:
sgd = LinearRegressionSGD(alpha=1e-4, max_steps=5000)
sgd.fit(X_train.values, y_train.values)
y_pred_sgd = sgd.predict(X_test.values)

print("MAPE для SGD:", MAPE(y_test, y_pred_sgd))

MAPE для SGD: nan


Часть 3 <br>Вычислите веса по точной формуле, используя X_train и y_train, предскажите с их помощью целевую переменную на X_test, записав результат в переменную y_pred_lr и вычислите ошибку MAPE

In [52]:
X_train_bias = np.hstack([np.ones((X_train.shape[0], 1)), X_train.values])
X_test_bias = np.hstack([np.ones((X_test.shape[0], 1)), X_test.values])

w_exact = np.linalg.pinv(X_train_bias.T @ X_train_bias) @ (X_train_bias.T @ y_train.values)
y_pred_lr = X_test_bias @ w_exact

print("MAPE для точного решения:", MAPE(y_test, y_pred_lr))

MAPE для точного решения: 31.758468351291373
